In [ ]:
import pandas as pd
import numpy as np
import zipfile
import math

In [ ]:
# Lettura del CSV iniziale
campaign_data_initial = pd.read_csv(zipfile.ZipFile('../data/campaign-data-initial.zip', 'r').open('../data/campaign-data-initial.csv'))

In [ ]:
campaign_data_processed = campaign_data_initial.copy()

### Cookie ID

In [ ]:
print('Numero di cookie ID erronei (negativi):', len(campaign_data_processed[campaign_data_processed['ad_form_id'] < 0]))

# Soluzione: utilizzare il valore assoluto del cookie ID
campaign_data_processed['ad_form_id'] = np.abs(campaign_data_processed['ad_form_id'])

print('Numero di cookie ID erronei dopo il processing:', len(campaign_data_processed[campaign_data_processed['ad_form_id'] < 0]))

# Check dei cookie ID duplicati
print("Numero di cookie ID duplicati:", len(campaign_data_processed) - len(campaign_data_processed.drop_duplicates()))

# Eliminazione della colonna dei cookie ID
campaign_data_processed.drop('ad_form_id', inplace = True, axis = 1)

### Suspicious

In [ ]:
# Cookie rilevati come sospetti bot
suspicious_cookie = campaign_data_processed[campaign_data_processed['suspicious'] == 1][['suspicious','clicks']]

# Check dei cookie sospetti con uno o più click
print('Cookie sospetti con uno o più click:', len(suspicious_cookie[suspicious_cookie['clicks'] >= 1]))

# Eliminazione della colonna suspicious
campaign_data_processed.drop('suspicious', inplace = True, axis = 1)

### Clicks

In [ ]:
print("Numero di cookie con uno o più click:", len(campaign_data_processed[(campaign_data_processed["clicks"] > 0)]))

### Impressions

In [ ]:
# Check: numero di impressions uguale a 0 (impossibile)
print('Numero di cookie con zero impressions (erronei):', len(campaign_data_processed[campaign_data_processed['impressions'] == 0 ]))
print('Numero di cookie con zero impressions ed almeno un click:', len(campaign_data_processed[ (campaign_data_processed['impressions'] == 0) & (campaign_data_processed['clicks'] > 0) ]))

# Somma del numero di click al numero di impression
campaign_data_processed.loc[campaign_data_processed.clicks >= 1,'impressions'] = campaign_data_processed['impressions'] + campaign_data_processed['clicks']

# Numero di impressions totali
initial_total_number_impressions = campaign_data_processed['impressions'].sum()

### Buy

In [ ]:
print("Numero di cookie che ha effettuato uno o più acquisti:", len(campaign_data_processed[campaign_data_processed['buy'] > 0]))

# Eliminazione della colonna buy
campaign_data_processed.drop('buy', inplace = True, axis = 1)

### Operating system

In [ ]:
# Check: cookie associato ad un unico sistema operativo (definizione di cookie)
os_check = campaign_data_processed.loc[:,'os_android':'os_windows'].sum(axis = 1)

print('Cookie con errore nel sistema operativo:', len(campaign_data_processed[os_check != 1])) 

In [ ]:
# Funzione che ritorna l'ID relativo al sistema operativo (ciascun SO ha un ID univoco)
def assignOsId(row):

    if row['os_android'] == 1:
        return 'os_A'

    if row['os_bsd'] == 1:
        return 'os_B'

    if row['os_ios'] == 1:
        return 'os_C'

    if row['os_linux'] == 1:
        return 'os_D'

    if row['os_osx'] == 1:
        return 'os_E'

    if row['os_other'] == 1:
        return 'os_F'

    if row['os_windows'] == 1:
        return 'os_G'

In [ ]:
# Creazione della colonna aggregata relativa al sistema operativo
campaign_data_processed.insert(

    loc = 0,
    column = 'os_id',
    value = campaign_data_processed.apply(lambda row: assignOsId(row), axis = 1)
    
)

In [ ]:
# Eliminazione delle colonne iniziali relative al sistema operativo
campaign_data_processed.drop(campaign_data_processed.loc[:,'os_android':'os_windows'], inplace = True, axis = 1)

### Browser

In [ ]:
# Check: cookie associato ad un unico browser (definizione di cookie)
browser_check = campaign_data_processed.loc[:,'browser_android':'browser_unknown'].sum(axis = 1)

print('Cookie con errore nel browser:', len(campaign_data_processed[browser_check != 1]))

In [ ]:
# Funzione che ritorna l'ID relativo al browser (ciascun browser ha un ID univoco)
def assignBrowserId(row):

    if row['browser_android'] == 1:
        return 'browser_A'

    if row['browser_chrome'] == 1:
        return 'browser_B'

    if row['browser_chromium'] == 1:
        return 'browser_C'

    if row['browser_edge'] == 1:
        return 'browser_D'

    if row['browser_firefox'] == 1:
        return 'browser_E'

    if row['browser_ie'] == 1:
        return 'browser_F'

    if row['browser_opera'] == 1:
        return 'browser_G'
    
    if row['browser_other'] == 1:
        return 'browser_H'
    
    if row['browser_safari'] == 1:
        return 'browser_I'
    
    if row['browser_unknown'] == 1:
        return 'browser_L'

In [ ]:
# Creazione della colonna aggregata relativa al browser
campaign_data_processed.insert(

    loc = 1,
    column = 'browser_id',
    value = campaign_data_processed.apply(lambda row: assignBrowserId(row), axis = 1)
    
)

In [ ]:
# Eliminazione delle colonne iniziali relative al browser
campaign_data_processed.drop(campaign_data_processed.loc[:,'browser_android':'browser_unknown'], inplace = True, axis = 1)

### Device type

In [ ]:
# Funzione che ritorna l'ID relativo al tipo di device (ciascun tipo di device ha un ID univoco)
def assignDeviceTypeId(row):

    if row['device_type'] == 1:
        return 'device_A'

    if row['device_type'] == 2:
        return 'device_B'

    if row['device_type'] == 3:
        return 'device_C'

    if row['device_type'] == 5:
        return 'device_D'

In [ ]:
# Creazione della colonna aggregata relativa al tipo di device
campaign_data_processed.insert(

    loc = 2,
    column = 'device_type_id',
    value = campaign_data_processed.apply(lambda row: assignDeviceTypeId(row), axis = 1)
    
)

In [ ]:
# Eliminazione della colonna iniziale relativa al tipo di device
campaign_data_processed.drop('device_type', inplace = True, axis = 1)

### Time

In [ ]:
# Check: numero di percentuali relative al momento della giornata con valore negativo
print("Numero di percentuali di tempo erronee (negative):", (campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'] < 0).values.sum())

# Soluzione: valore assoluto delle percentuali
campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'] = np.abs(campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'])

In [ ]:
# Check: somma delle percentuali relative al momento della giornata

# Colonna con la somma delle percentuali relative al momento della giornata
campaign_data_processed.insert(
    
    loc = 13, 
    column = 'time_sum',
    value = campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'].sum(axis = 1)
    
)

print('Numero di osservazioni erronee (problema somma percentuali diverse da 100):', len(campaign_data_processed[campaign_data_processed['time_sum'] != 100]))

# Soluzione: normalizzazione basata sulla somma della riga

# Funzione che normalizza le percentuali della singola riga basandosi sul totale (ritorna un ratio nel range [0,1])
def normalizePercentage(row):
    return round(row[0:-1] / row[-1], 2)

# Applicazione della normalizzazione
campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'] = campaign_data_processed.loc[:,'time1_workday_morning':'time_sum'].apply(lambda row: normalizePercentage(row), axis = 1)

# Eliminazione della colonna temporanea della somma
campaign_data_processed.drop('time_sum', inplace = True, axis = 1)

In [ ]:
# Salvataggio della distribuzione iniziale dei ratio per momento della giornata (utile per calcolare la correttezza di MIDA)
initialRatioDistribution = campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'].copy()

# Rinomina delle colonne relative alle impression per momento della giornata nella distribuzione iniziale
initialRatioDistribution = initialRatioDistribution.rename(columns = {

    'time1_workday_morning': 'impressions_workday_morning',
    'time1_workday_afternoon': 'impressions_workday_afternoon',
    'time1_workday_evening': 'impressions_workday_evening',
    'time1_workday_night': 'impressions_workday_night',
    'time1_weekend_morning': 'impressions_weekend_morning',
    'time1_weekend_afternoon': 'impressions_weekend_afternoon',
    'time1_weekend_evening': 'impressions_weekend_evening',
    'time1_weekend_night': 'impressions_weekend_night'

})

In [ ]:
# Algoritmo: Missing Impressions Detection Algorithm (MIDA)
# Params
#   @row -> riga del dataset con il numero di impressions iniziale ed i ratio per ciascun momento della giornata
#   @threshold -> soglia minima unità impressions
def mida(row, threshold):

    # Se un momento della giornata ha il 100% delle impressions, allora ritorna il numero delle impressions stesso
    if(len((row['time1_workday_morning':].where(row['time1_workday_morning':] == 1)).dropna()) == 1):

        return row
    
    else:


        # Lunghezza della distribuzione iniziale di valori ratio maggiori di 0
        initialDistributionLength = len(row['time1_workday_morning':][row['time1_workday_morning':] > 0])


        # Ratio sotto la soglia minima di unità d'impression vengono posti uguali a 0
        row['time1_workday_morning':][row['time1_workday_morning':] < threshold] = 0
        

        # Check sulla lunghezza della distribuzione filtrata con threshold e la lunghezza della distribuzione iniziale (per normalizzazione)
        if(len(row['time1_workday_morning':][row['time1_workday_morning':] > 0]) < initialDistributionLength):
            
            # Normalizzazione nel range [0,1]
            row['time1_workday_morning':] = round(row['time1_workday_morning':] / row['time1_workday_morning':].sum(), 2)


        # Unità d'impression è il minimo tra i ratio maggiori di 0
        impressionUnit = row['time1_workday_morning':][row['time1_workday_morning':] > 0].min()

        # Numero di impressions calcolato dall'algoritmo
        impressions_mida = 1 / impressionUnit

        
        # Arrotondamento per eccesso da .5 compreso, altrimenti per difetto
        if(impressions_mida - math.floor(impressions_mida) < 0.5): impressions_mida = math.floor(impressions_mida)
        else: impressions_mida = math.ceil(impressions_mida)


        # Utilizziamo impressions_mida solamente se vengono trovate più impressions rispetto all'inizio
        if(impressions_mida >= row['impressions']):

            row['impressions'] = impressions_mida

        return row

In [ ]:
# Colonna con il numero di impressions calcolate dall'algoritmo MIDA
campaign_data_processed.loc[:,'impressions':'time1_weekend_night'] = campaign_data_processed.loc[:,'impressions':'time1_weekend_night'].apply(lambda row: mida(row, 0.05), axis = 1)

# Rinomina della colonna delle impressions
campaign_data_processed = campaign_data_processed.rename(columns = { 'impressions': 'impressions_mida' })

In [ ]:
# Analisi del numero di impressions iniziali e di quelle computate con MIDA
mida_total_number_impressions = campaign_data_processed['impressions_mida'].sum()

difference_number_impressions = mida_total_number_impressions - initial_total_number_impressions

avg_number_impressions = mida_total_number_impressions / len(campaign_data_processed)
max_number_impressions = campaign_data_processed['impressions_mida'].max()

print('Numero totale di impressions iniziali:', initial_total_number_impressions)
print('Numero totale di impressions calcolate con MIDA:', int(mida_total_number_impressions))
print('Numero di nuove impressions rilevate da MIDA:', int(difference_number_impressions))
print('Numero medio di MIDA impressions per cookie:', round(avg_number_impressions, 1))

In [ ]:
# Funzione che calcola il numero di impressions per ciascun momento della giornata (totale impressions * ratio)
def computeImpressionsPerDayMoment(row):

    impressions = row['impressions_mida']
    impressionsPerDayMoment = round(impressions * row['time1_workday_morning':'time1_weekend_night']).apply(int)
    sumImpressionsPerDayMoment = impressionsPerDayMoment.sum()
    
    # Gestione delle differenze tra impression calcolate da MIDA e la somma di quelle calcolate coi ratio (problema arrotondamento)
    if(sumImpressionsPerDayMoment != impressions):

        diff = sumImpressionsPerDayMoment - impressions

        if(diff > 0):

            while(diff != 0):

                diff = diff - 1
                randColumn = impressionsPerDayMoment.loc[lambda x: x > 0].sample(n = 1).index[0]
                impressionsPerDayMoment[randColumn] = impressionsPerDayMoment[randColumn] - 1

        else:

            while(diff != 0):

                diff = diff + 1
                randColumn = impressionsPerDayMoment.loc[lambda x: x > 0].sample(n = 1).index[0]
                impressionsPerDayMoment[randColumn] = impressionsPerDayMoment[randColumn] + 1

    return impressionsPerDayMoment

# Calcolo delle impressions per momento della giornata
campaign_data_processed.loc[:,'time1_workday_morning':'time1_weekend_night'] = campaign_data_processed.loc[:,'impressions_mida':'time1_weekend_night'].apply(lambda row: computeImpressionsPerDayMoment(row), axis = 1)

# Conversione in tipo int
campaign_data_processed.loc[:,'impressions_mida':'time1_weekend_night'] = campaign_data_processed.loc[:,'impressions_mida':'time1_weekend_night'].apply(lambda row: row.apply(int))

In [ ]:
# Rinomina delle colonne relative alle impressions per momento della giornata
campaign_data_processed = campaign_data_processed.rename(columns = {

    'impressions_mida': 'total_impressions',
    'time1_workday_morning': 'impressions_workday_morning',
    'time1_workday_afternoon': 'impressions_workday_afternoon',
    'time1_workday_evening': 'impressions_workday_evening',
    'time1_workday_night': 'impressions_workday_night',
    'time1_weekend_morning': 'impressions_weekend_morning',
    'time1_weekend_afternoon': 'impressions_weekend_afternoon',
    'time1_weekend_evening': 'impressions_weekend_evening',
    'time1_weekend_night': 'impressions_weekend_night'

})

In [ ]:
# Eliminazione delle colonne relative al momento della giornata con granularità maggiore
campaign_data_processed.drop(campaign_data_processed.loc[:,'time2_morning_early':'time2_sleep'].columns, inplace = True, axis = 1)

In [ ]:
# Funzione che calcola la correttezza percentuale di MIDA per ciascuna riga
def computeAccuracyMIDA(row):

    newDistributionMida = round(row['impressions_workday_morning':'impressions_weekend_night'] / row['total_impressions'], 2)
    distance = (newDistributionMida - initialRatioDistribution.iloc[row.name,:]).abs().sum()

    return (1 - round(distance, 2)) * 100

In [ ]:
# Correttezza dell'algoritmo MIDA
accuracyMIDA = campaign_data_processed.loc[:,'total_impressions':'impressions_weekend_night'].apply(lambda row: computeAccuracyMIDA(row), axis = 1)

print("Correttezza media MIDA:", round(accuracyMIDA.mean(), 2), "%")
print("Correttezza mediana MIDA:", accuracyMIDA.median(), "%")

### Lunghezza del testo della pagina

In [ ]:
# Dataframe con le colonne relative alla lunghezza del testo
page_length = campaign_data_processed.loc[:,'L00_50':'L10001_more']

In [ ]:
# Pre-processing sui NaN, inf e valori negativi come riportato nel paper
page_length = page_length.fillna(0).replace([np.inf, -np.inf], 100).abs()

In [ ]:
# Dataframe con osservazioni con tutte le colonne uguali a 0
page_length_erroneus = page_length[(page_length.isin([0]).sum(axis = 1)) == 9]

# Dataframe con osservazioni corrette
page_length_ok = page_length[(page_length.isin([0]).sum(axis = 1)) < 9]

# Numero di osservazioni con tutte le colonne uguali a 0
print('Numero di righe erronee:', len(page_length_erroneus))

# Calcolo della mediana per ciascuna colonna
print('Mediana per ciascuna colonna\n', page_length_ok.median())

# Aggiustamento delle righe erronee come spiegato nel paper (L00_50 = 1)
page_length.loc[page_length_erroneus.index,'L00_50'] = 1

In [ ]:
# Normalizzazione delle osservazioni nel range [0,1]
page_length = page_length.apply(lambda row: round(row / row.sum(), 2), axis = 1)

In [ ]:
# Percentuali di osservazioni con specifiche colonne uguali a 0
# Colonna L00_50
print('L00_50:', round((len(page_length[page_length.iloc[:,0] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L51_100
print('L51_100:', round((len(page_length[page_length.iloc[:,1] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L101_250
print('L101_250:', round((len(page_length[page_length.iloc[:,2] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L251_500
print('L251_500:', round((len(page_length[page_length.iloc[:,3] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L501_1000
print('L501_1000:', round((len(page_length[page_length.iloc[:,4] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L1001_2500
print('L1001_2500:', round((len(page_length[page_length.iloc[:,5] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L2501_5000
print('L2501_5000:', round((len(page_length[page_length.iloc[:,6] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L5001_10000
print('L5001_10000:', round((len(page_length[page_length.iloc[:,7] == 0]) / len(page_length)) * 100, 2), '%')

# Colonna L10001_more
print('L10001_more:', round((len(page_length[page_length.iloc[:,8] == 0]) / len(page_length)) * 100, 2), '%')

In [ ]:
# Aggregazione delle colonne con lunghezza del testo maggiore di 250 caratteri
page_length['L251_more'] = page_length.loc[:,'L251_500':'L10001_more'].sum(axis = 1)

# Eliminazione delle colonne precedentemente aggregate
page_length.drop(page_length.loc[:,'L251_500':'L10001_more'].columns, inplace = True, axis = 1)

In [ ]:
page_length['impressions'] = campaign_data_processed['total_impressions']

In [ ]:
def computeImpressionsPerPageLength(row):
    
    impressions = row['impressions']

    # Gestione della problematica delle impressions con percentuali sbagliate
    if(impressions == 1): 
        
        index_max = row['L00_50':'L251_more'].idxmax()
        row = row.apply(lambda x: 0)
        row[index_max] = 1

        return row[:-1].apply(int)

    else:

        impressionsPerPageLength = round(impressions * row['L00_50':'L251_more']).apply(int)
        sumImpressionsPerPageLength = impressionsPerPageLength.sum()

        if(sumImpressionsPerPageLength != impressions):

            diff = sumImpressionsPerPageLength - impressions

            if(diff > 0):

                while(diff != 0):

                    diff = diff - 1
                    randColumn = impressionsPerPageLength.loc[lambda x: x > 0].sample(n = 1).index[0]
                    impressionsPerPageLength[randColumn] = impressionsPerPageLength[randColumn] - 1

            else:

                while(diff != 0):

                    diff = diff + 1
                    randColumn = impressionsPerPageLength.loc[lambda x: x > 0].sample(n = 1).index[0]
                    impressionsPerPageLength[randColumn] = impressionsPerPageLength[randColumn] + 1

        return impressionsPerPageLength

In [ ]:
page_length = page_length.apply(lambda row: computeImpressionsPerPageLength(row), axis = 1)

In [ ]:
campaign_data_processed.insert(
    
    loc = 16, 
    column = 'L251_more',
    value = 0
    
)

In [ ]:
campaign_data_processed.loc[:,'L00_50':'L251_more'] = page_length

In [ ]:
# Eliminazione delle vecchie colonne relative alla lunghezza del testo
campaign_data_processed.drop(campaign_data_processed.loc[:,'L251_500':'L10001_more'].columns, inplace = True, axis = 1)

# Rinomina delle colonne relative alla lunghezza del testo
campaign_data_processed = campaign_data_processed.rename(columns = {

    'L00_50': 'impressions_page_length_1',
    'L51_100': 'impressions_page_length_2',
    'L101_250': 'impressions_page_length_3',
    'L251_more': 'impressions_page_length_4'

})

### Categorie

In [ ]:
# campaign_data_processed.to_csv('campaign_preprocessed_partial.csv', index=False)
# campaign_data_processed = pd.read_csv(zipfile.ZipFile('campaign_preprocessed_partial.zip', 'r').open('campaign_preprocessed_partial.csv'))

In [ ]:
# Eliminazione delle colonne relative alle categorie di livello 2 e 3
campaign_data_processed.drop(campaign_data_processed.loc[:,'categories2_accessories':'categories3_zoology'].columns, inplace = True, axis = 1)

In [ ]:
# Rinomina delle colonne delle categorie di livello 1
def newName(name):
    return name.replace(name, name.split('_')[1])

campaign_data_processed = pd.concat([ campaign_data_processed, campaign_data_processed.loc[:,'categories1_artandentertainment':'categories1_uncategorized'].rename(columns = newName) ], axis = 1)
campaign_data_processed.drop(campaign_data_processed.loc[:,'categories1_artandentertainment':'categories1_uncategorized'].columns, inplace = True, axis = 1)

In [ ]:
# Dataframe con le colonne relative alle categorie di livello 1
campaign_category_1 = campaign_data_processed.loc[:,'artandentertainment':'uncategorized']

In [ ]:
# Aggiustamento dei valori NaN, inf e negativi
campaign_category_1 = campaign_category_1.fillna(0).replace([np.inf, -np.inf], 100).abs()

In [ ]:
# Elimino le colonne che hanno somma uguale a 0, ovvero non assumono alcun valore -> emotions (categoria non riconosciuta)
campaign_category_1.drop(campaign_category_1.sum()[campaign_category_1.sum() == 0].index, inplace = True, axis = 1)

In [ ]:
# Dataframe con osservazioni che hanno categoria di livello 1
cat1_ok = campaign_category_1[campaign_category_1.sum(axis = 1) != 0]

# Normalizzazione nel range [0,1]
cat1_ok = cat1_ok.apply(lambda row: round(row/row.sum(), 2), axis = 1)

# Inserimento delle colonne normalizzate nel dataframe
campaign_category_1.loc[cat1_ok.index,:] = cat1_ok

In [ ]:
# Check: numero di cookie senza categoria di livello 1 
cat1_check = campaign_category_1.sum(axis = 1)

print('Numero di cookie senza categoria di livello 1:', len(campaign_category_1[cat1_check == 0]))

# Soluzione: aggiustamento delle righe erronee mettendo la media di ciascuna colonna di tutte le altre righe
# Riga con le medie (computata sulle categorie ok)
mean_category = round(cat1_ok.mean(), 2)

# Sostituzione delle righe erronee
campaign_category_1[cat1_check == 0] = campaign_category_1[cat1_check == 0].apply(lambda row: mean_category, axis = 1)

In [ ]:
# Idea: splitting del ratio di uncategorized in tutte le altre categorie (ed eliminazione di uncategorized)

# Funzione che distribuisce il valore di uncategorized sulle altre colonne 
def distributeUncategorized(row):

    if(row['uncategorized'] != 0.00):

        n = row['uncategorized'] / (len(row) - 1)
        return round(row + n, 2)
    
    else:

        return row

# Distribuzione del valore di uncategorized sulle altre colonne
campaign_category_1 = campaign_category_1.apply(lambda row: distributeUncategorized(row), axis = 1)

# Rimozione della colonna uncategorized
campaign_category_1.drop('uncategorized', inplace=True, axis=1)

In [ ]:
# Nuova normalizzazione nel range [0,1]
campaign_category_1 = campaign_category_1.apply(lambda row: round(row/row.sum(),2), axis = 1)

In [ ]:
# Idea: considerare range di ratio (eliminare variabilità e ottimizzazione clustering)

# Funzione che ritorna l'ID relativo al range
''''
def assignRange(row):

    i = 0
    
    for element in row:
        
        if(element == 0.00):
           row[row.index[i]] = 0
           
        if((element > 0) & (element <= 0.1)):
            row[row.index[i]] = 1
            
        if((element > 0.1) & (element <= 0.2)):
            row[row.index[i]] = 2
        
        if((element > 0.2) & (element <= 0.3)):
            row[row.index[i]] = 3
            
        if((element > 0.3) & (element <= 0.4)):
            row[row.index[i]] = 4
        
        if((element > 0.4) & (element <= 0.5)):
            row[row.index[i]] = 5
        
        if((element > 0.5) & (element <= 0.6)):
            row[row.index[i]] = 6
        
        if((element > 0.6) & (element <= 0.7)):
            row[row.index[i]] = 7
        
        if((element > 0.7) & (element <= 0.8)):
            row[row.index[i]] = 8
        
        if((element > 0.8) & (element <= 0.9)):
            row[row.index[i]] = 9
        
        if((element > 0.9) & (element <= 1)):
            row[row.index[i]] = 10
        
        i = i + 1
    
    return row.apply(int)

# Assegnazione dei range a tutti i valori delle colonne delle categorie di livello 1
campaign_category_1 = campaign_category_1.apply(lambda row: assignRange(row), axis = 1)

'''

In [ ]:
# Rimozione delle colonne emotions ed uncategorized dal dataframe principale
campaign_data_processed.drop('emotions', inplace = True, axis = 1)
campaign_data_processed.drop('uncategorized', inplace=True, axis=1)

campaign_data_processed.loc[:,'artandentertainment':'travel'] = campaign_category_1

### Admants

In [ ]:
# Check: osservazioni con tutte le colonne relative ad admants uguali a 0
admants_check = campaign_data_processed.loc[:,'admants1_appliances':'admants1_videoandcomputergames'].sum(axis = 1)

print('Numero di cookie senza admants:', len(campaign_data_processed[admants_check == 0]))

# Eliminazione delle colonne relative ad admants
campaign_data_processed.drop(campaign_data_processed.loc[:,'admants1_appliances':'admants1_videoandcomputergames'].columns, inplace = True, axis = 1)

### Sentiments

In [ ]:
# Check: numero di cookie senza sentiments
sentiments_check = campaign_data_processed.loc[:,'sentiments1_neutroneutralsentiment':'sentiments1_sentimentpositivesentiment'].sum(axis = 1)

print('Numero di cookie senza sentiments:', len(campaign_data_processed[sentiments_check == 0]))

# Eliminazione delle colonne relative ai sentiments
campaign_data_processed.drop(campaign_data_processed.loc[:,'sentiments1_neutroneutralsentiment':'sentiments1_sentimentpositivesentiment'].columns, inplace = True, axis = 1)

### Feelings

In [ ]:
# Check: numero di cookie senza feelings
feelings_check = campaign_data_processed.loc[:,'feelings1_active':'feelings1_wealth'].sum(axis = 1)

print('Numero di cookie senza feelings:', len(campaign_data_processed[feelings_check == 0]))

# Eliminazione delle colonne relative ai feelings
campaign_data_processed.drop(campaign_data_processed.loc[:,'feelings1_active':'feelings1_wealth'].columns, inplace = True, axis = 1)

### Dataset pre-processed

In [ ]:
campaign_data_processed.to_csv('../data/campaign-data-processed.csv', index = False)